In [2]:
demand = [0, 34, 12, 65, 10, 43, 27, 55, 61, 22]
sites = [(15, 38), (23, -19), (44, 62), (3, 12), (-56, -21), (-53, 2),
         (33, 63), (14, -33), (42, 41), (13, -62)]

In [13]:
len(demand)

10

In [14]:
len(sites)

10

In [4]:
from dwave.optimization.generators import capacitated_vehicle_routing
model = capacitated_vehicle_routing(
    demand=demand,
    number_of_vehicles=2,
    vehicle_capacity=200,
    locations_x=[x for x,y in sites],
    locations_y=[y for x,y in sites])

In [5]:
from dwave.system import LeapHybridNLSampler
sampler = LeapHybridNLSampler()                  

In [6]:
print(sampler.estimated_min_time_limit(model))

5


In [7]:
results = sampler.sample(
    model,
    time_limit=10)          

In [8]:
print(results.result().timing['charge_time'])       

10000000


In [12]:
results.result()

SampleResult(model=<dwave.optimization.model.Model object at 0x10733db10>, timing={'qpu_access_time': 762352, 'warnings': [], 'charge_time': 10000000, 'run_time': 10027969})

In [9]:
num_samples = model.states.size()
route, = model.iter_decisions()                     
route1, route2 = route.iter_successors()            
for i in range(min(3, num_samples)):
    print(f"Objective value {int(model.objective.state(i))} for \n" \
    f"\t Route 1: {route1.state(i)} \t Route 2: {route2.state(i)} \n" \
    f"\t Feasible: {all(sym.state(i) for sym in model.iter_constraints())}")   

Objective value 423 for 
	 Route 1: [4. 3. 8. 6. 0.] 	 Route 2: [2. 7. 1. 5.] 
	 Feasible: True
Objective value 423 for 
	 Route 1: [0. 6. 8. 3. 4.] 	 Route 2: [5. 1. 7. 2.] 
	 Feasible: True
Objective value 423 for 
	 Route 1: [0. 6. 8. 3. 4.] 	 Route 2: [2. 7. 1. 5.] 
	 Feasible: True


In [ ]:
model.states.resize(1)
route.set_state(0, [route1.state(0), route2.state(0)])   
results = sampler.sample(model)     

In [1]:
import shutil
with open("model_states", 'wb') as f:                    
    shutil.copyfileobj(model.states.to_file(), f)